Multi-Tool Agent Assistant (Web Search + Calculator + Weather)
Create an AI assistant that uses tools to answer complex questions with tools for Web Search (SerpAPI), Calculator, Weather (API). Ask: "What’s the weather in Paris and what’s 5°C in Fahrenheit?"

In [1]:
import os
from langchain_groq import ChatGroq
from langchain_core.tools import tool
from langchain_community.utilities.serpapi import SerpAPIWrapper
from langchain.agents import create_tool_calling_agent, AgentExecutor
from langchain.prompts import ChatPromptTemplate
from langchain.memory import ConversationSummaryBufferMemory


from dotenv import load_dotenv
load_dotenv()
groq = os.getenv("GROQ_API_KEY")
serp = os.getenv("SERPAPI_API_KEY")

llm = ChatGroq(model="llama3-70b-8192", temperature=0)


SerpAPI tool

In [2]:
search = SerpAPIWrapper()

@tool
def web_search(query: str) -> str:
    """Search the web for current information using SerpAPI."""
    return search.run(query)


Calculator tool


In [3]:
@tool
def calc(operation: str, a: float, b: float) -> float:
    """Perform basic arithmetic. Operation must be 'add' or 'subtract'."""
    if operation == "add":
        return a + b
    elif operation == "subtract":
        return a - b
    elif operation == "multiply":
        return a * b
    elif operation == "divide":
        return a / b
    else:
        raise ValueError("Unsupported operation. Use 'add','subtract','multiplication','division'.")


Celsius to ahrenheit tool

In [4]:
@tool
def celsius_to_fahrenheit(celsius: float) -> float:
    """Convert Celsius to Fahrenheit."""
    return (celsius * 9/5) + 32

Defining the tools and prompt for model

In [5]:
tools = [web_search, calc, celsius_to_fahrenheit]

prompt = ChatPromptTemplate.from_messages([
    ("system",
     "You are a helpful AI assistant. You can use the following tools:\n"
     "- 'web_search': to search for current information like weather.\n"
     "- 'celsius_to_fahrenheit': to convert Celsius to Fahrenheit.\n"
     "- 'calculator': to perform basic math like addition or multiplication.\n"
     "Respond step by step and invoke tools as needed."),
    ("user", "{input}"),
    ("placeholder", "{agent_scratchpad}")
])


Initializing the agent and memory

In [6]:
from langchain.agents import AgentExecutor, create_tool_calling_agent
from langchain.memory import ConversationBufferMemory

agent = create_tool_calling_agent(
    llm=llm,
    tools=tools,
    prompt=prompt
)


memory = ConversationBufferMemory(
    llm=llm,
    max_token_limit=1000,
    return_messages=True
)


agent_executor = AgentExecutor.from_agent_and_tools(
    agent=agent,
    tools=tools,
    memory=memory,
    handle_parsing_errors=True,
)

C:\Users\sayhe\AppData\Local\Temp\ipykernel_1952\2680405380.py:11: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(


Testing

In [7]:
response = agent_executor.invoke({
    "input": "What's the current weather in Paris? Also, convert 5 degrees Celsius to Fahrenheit and also calculate the sum of 10 and 42."
})
print(response["output"])


The current weather in Paris is mostly cloudy with a temperature of 68 degrees Fahrenheit, 0% precipitation, 56% humidity, and a wind speed of 5 mph. 5 degrees Celsius is equal to 41.0. The sum of 10 and 42 is 52.0.
